# KDTree Test

One of the slowest parts (now that multiprocessing is working) is the pruning of extraneous peaks. It seems to me that there are 3 ways we can make this go faster.

1. Brute force (current implementation) + `multiprocessing`
2. Dynamic programming, we can build up a solution one peak at a time
3. We could use KDTrees to find nearest neighbors quickly and then brute force our way through those

The last two seem to be the best choices, from a learning prospective an elegance prospective and probably and efficiency prospective.

One important thing to note is that the highest amplitude peak should be analyzed first

In [1]:
import numpy as np
import itertools as itt
from numpy.linalg import norm
from scipy.spatial import KDTree

In [2]:
def prune_blobs(blobs, diameter):
    """Original implementation of method, taken from skimage.
    This is a simple, and naive brute force calculation.
    """
    # assumes blobs have structure of y0, x0, width, amp
    # make a copy of blobs otherwise it will be changed
    temp_blobs = blobs.copy()
    # cycle through all possible pairwise cominations of blobs
    for blob1, blob2 in itt.combinations(temp_blobs, 2):
        # take the norm of the difference in positions and compare
        # with diameter
        if norm((blob1 - blob2)[0:2]) < diameter:
            # compare intensities and use the third column to keep
            # track of which blobs to toss
            if blob1[3] > blob2[3]:
                blob2[3] = -1
            else:
                blob1[3] = -1

    # set internal blobs array to blobs_array[blobs_array[:, 2] > 0]
    new_blobs = np.array([
        a for b, a in zip(temp_blobs, blobs) if b[3] > 0
    ])

    # Return a copy of blobs incase user wants a onliner
    return new_blobs

In [ ]:
# make fake data